# Laboratorio 1 - Estructuras de Datos II

## Imports y Clases

In [21]:
import graphviz as gv
import geopandas as geo
from pprint import pprint
from typing import Any, List, Optional, Tuple
import pandas as pd
import seaborn as sns


def preorder(node: Optional["Node"]) -> None:
    if node is not None:
        print(node.data, end=' ')
        preorder(node.left)
        preorder(node.right)

def inorder(node: Optional["Node"]) -> None:
    if node is not None:
        inorder(node.left)
        print(node.data, end=' ')
        inorder(node.right)

def postorder(node: Optional["Node"]) -> None:
  if node is not None:
      postorder(node.left)
      postorder(node.right)
      print(node.data, end=' ')

class Height:
    def __init__(self):
        self.height = 0

def isHeightBalanced(root, height):

    left_height = Height()
    right_height = Height()

    if root is None:
        return True

    l = isHeightBalanced(root.left, left_height)
    r = isHeightBalanced(root.right, right_height)

    height.height = max(left_height.height, right_height.height) + 1

    if abs(left_height.height - right_height.height) <= 1:
        return l and r

    return False


class Stack:

    def __init__(self, size: int) -> None:
        self.stack: List[Any] = []
        self.size = size

    def __repr__(self) -> str:
        return str(self.stack)

    def add(self, elem: Any) -> None:
        if len(self.stack) >= self.size:
            raise ValueError('The Stack is full')

        self.stack.append(elem)

    def remove(self) -> Any:
        if not self.stack:
            raise ValueError('The Stack is empty')

        return self.stack.pop()

    def is_empty(self) -> bool:
        return len(self.stack) == 0

class Queue:

    def __init__(self, size: int) -> None:
        self.queue: List[Any] = []
        self.size = size

    def __repr__(self) -> str:
        return str(self.queue)

    def add(self, elem: Any) -> None:
        if len(self.queue) >= self.size:
            raise ValueError('The Queue is full')

        self.queue.append(elem)

    def remove(self) -> Any:
        if not self.queue:
            raise ValueError('The Queue is empty')

        return self.queue.pop(0)

    def is_empty(self) -> bool:
        return len(self.queue) == 0

class Node:

    def __init__(self, data: Any, lvl=0) -> None:
        self.data = data
        self.lvl = lvl
        self.parent: Optional["Node"] = None
        self.left: Optional["Node"] = None
        self.right: Optional["Node"] = None

    def __repr__(self) -> str:
        left = self.left.data if self.left is not None else None
        right = self.right.data if self.right is not None else None
        return f'{self.data} {left} {right}'


class AVL:

    def __init__(self) -> None:
        self.root: Optional["Node"] = None

    def search(self, elem: Any) -> Tuple[Optional["Node"], Optional["Node"]]:
        p, pad = self.root, None
        while p is not None:
            if elem == p.data:
                return p, pad
            else:
                pad = p
                if elem < p.data:
                    p = p.left
                else:
                    p = p.right

        return p, pad

    def insert(self, elem: Any) -> bool:
        to_insert = Node(elem)
        if self.root is None:
            self.root = to_insert
            return True
        else:
            p, pad = self.search(elem)
            if p is None:
                if elem < pad.data:
                    pad.left = to_insert
                    pad.left.lvl = pad.lvl + 1
                    pad.left.parent = pad
                else:
                    pad.right = to_insert
                    pad.right.lvl = pad.lvl + 1
                    pad.right.parent = pad
                return True
            return False

    def delete(self, elem: Any, mode: bool = True) -> bool:
        p, pad = self.search(elem)
        if p is not None:
            if p.left is None and p.right is None:
                if p == pad.left:
                    pad.left = None
                else:
                    pad.right = None
                del p
            elif p.left is None and p.right is not None:
                if p == pad.left:
                    pad.left = p.right
                else:
                    pad.right = p.right
                del p
            elif p.left is not None and p.right is None:
                if p == pad.left:
                    pad.left = p.left
                else:
                    pad.right = p.left
                del p
            else:
                if mode:
                    pred, pad_pred = self.__pred(p)
                    p.data = pred.data
                    if pred.left is not None:
                        pad_pred.right = pred.left
                    else:
                        pad_pred.right = None
                    del pred
                else:
                    sus, pad_sus = self.__sus(p)
                    p.data = sus.data
                    if sus.right is not None:
                        pad_sus.left = sus.right
                    else:
                        pad_sus.left = None
                    del sus

            return True
        return False

    def __pred(self, node: "Node") -> Tuple["Node", "Node"]:
        p, pad = node.left, node
        while p.right is not None:
            p, pad = p.right, p

        return p, pad

    def __sus(self, node: "Node") -> Tuple["Node", "Node"]:
        p, pad = node.right, node
        while p.left is not None:
            p, pad = p.left, p

        return p, pad

    def __inorder(self, node: Optional["Node"]) -> None:
        if node is not None:
            self.__inorder(node.left)
            print(node)
            self.__inorder(node.right)

    def inorder(self) -> None:
        self.__inorder(self.root)

    def left_rotate(self, node: "Node") -> "Node":
        new_root = node.right
        node.right = new_root.left
        new_root.left = node
        return new_root

    def right_rotate(self, node: "Node") -> "Node":
        new_root = node.left
        node.left = new_root.right
        new_root.right = node
        return new_root

    def height(self, node):
        if node is None:
          return 0
        else:
          return max(self.height(node.left), self.height(node.right)) + 1


## CSV

In [2]:
df = pd.read_csv("co_properties_final.csv")
df.head()

,title,department,city,property_type,latitude,longitude,surface_total,surface_covered,bedrooms,bathrooms,operation_type,price
0,Apartamento En Arriendo/venta En Barranquilla ...,Atlántico,Barranquilla,Apartamento,11.013,-74.836,70.0,70.0,1.0,2.0,Venta,250000000.0
1,Casa En Venta En Cali Ciudad Jardn Cod. VVLZ3039,Valle del Cauca,Cali,Casa,3.368,-76.531,420.0,420.0,5.0,5.0,Venta,950000000.0
2,Casa En Arriendo/venta En Barranquilla Bellavi...,Atlántico,Barranquilla,Casa,10.997,-74.791,83.0,150.0,3.0,3.0,Venta,320000000.0
3,Casa En Arriendo En Chia Chia Cod. AINH2992,Cundinamarca,Chía,Casa,4.845,-74.057,580.0,2.0,8.0,3.0,Arriendo,8000000.0
4,Apartamento En Arriendo En Cali Ciudad Crdoba ...,Valle del Cauca,Cali,Apartamento,3.402,-76.507,56.0,56.0,3.0,1.0,Arriendo,550000.0
